# Steady State Solution By Finite Difference Method.

We need to solve

$$
    \newcommand{\qty}[1]
    {
        \left({#1}\right)
    }
    \newcommand{\qtys}[1]
    {
        \left[{#1}\right]
    }
$$     

\begin{align}
\label{eq:adimetional-diffusion-nernst}
    \nabla \cdot \qty{\nabla \rho_+ - \rho_+ \nabla \Psi} &= 0, \\
    \frac{\mathcal{D}_-}{\mathcal{D_+}}\nabla \cdot \qty{\nabla \rho_- + \rho_- \nabla \Psi}  &= 0, \\
    \nabla^2 \Psi &= \kappa^2\qty{\rho_- - \rho_+}.
\end{align}


With boundary conditions

\begin{align}
    J_+(\xi = 0) &= -\mathcal{D}_+\kappa C_b\qty{\frac{\partial \rho_+}{\partial \xi} - \rho_+ \frac{\partial\Psi}{\partial \xi}}\bigg|_{x= 0}= -k_f C_b\rho_+(\xi = 0, \tau)\\
    J_-(\xi = 0) &= -\mathcal{D}_-\kappa C_b\qty{\frac{\partial \rho_-}{\partial x} + \rho_- \frac{\partial\Psi}{\partial \xi}}  \bigg|_{x= 0} = 0\\
    \rho_+(\delta) = 1\\
    \rho_-(\delta) = 1\\
    \Psi(\xi = 0) &= \frac{z\mathcal{F}}{RT} V_0 = \Psi_0\\
    \frac{\partial\Psi}{\partial x}(\xi = 1) &= 0
\end{align}






Integrating equations 1 and 2 we get 

\begin{align}
    \qty{\nabla \rho_+ - \rho_+ \nabla \Psi} = A_+\\
    \qty{\nabla \rho_+ - \rho_+ \nabla \Psi} = A_-
\end{align}

where $A_s$ are constants. To determine these constants we use boundary conditions. We get

\begin{align}
A_+ = \frac{k_f}{\kappa D_+}\rho(\xi=0) \\
A_- = 0
\end{align}

In [1]:
#imports
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import diags
import math
from decimal import Decimal

In [2]:
class Model():
    def __init__(self, params):
        #Model Parameters
        self.Cb = params['bulkConcentration']
        self.D = params['diffusionCoefficientScale']
        self.d = params['laminarFlowRegion']
        self.kf = params['reactionRate']
        self.z = params['z']
        self.F = params['Fa']
        self.R = params['R']
        self.T = params['T']
        self.V0 = params['V0']
        self.epsilon = params['epsilon']
        self.kappa =  np.sqrt(( ( self.z * self.F  ) ** 2 * self.Cb ) / ( self.epsilon * self.R * self.T ) )
        self.Psi0 = self.z * self.F * params['V0'] / ( self.R * self.T )
        self.D1 = self.D * params["D1"]
        self.D2 = self.D * params["D2"]
        self.M = 100
        self.length = params["length"]
        self.xi = np.linspace(0,params["length"], self.M)
    

        #Grid Parameters
        self.dxi = self.length/(self.M) # M Partitions 

        #Plotting parameters
        self.imageName = 'complete-diffusion-nernst'
        
    def build(self):
        M = self.M
        Psi0 = self.Psi0
        kappa = self.kappa
        kf = self.kf
        dxi = self.dxi
        D1 = self.D1 
        D2 = self.D2
        # Define the coefficient matrix
        g1 = 1 / ( 1 + kf * dxi / ( D1 * kappa ) +  Psi0)
        di1 = ( 1 - 2 * a1 ) * np.ones(M-2)
        di1[0] = ( 1 - 2 * a1 + a1 * g1 )
        A1 = diags(np.array([ a1 * np.ones(M-3), di1, a1 * np.ones(M-3)]), [-1, 0, 1], shape=(M-2, M-2)).toarray()

        g2 = 1 / ( 1 - Psi0 )
        di2 = ( 1 - 2 * a2 ) * np.ones(M-2)
        di2[0] = ( 1 - 2 * a2 + a2 * g2 )
        A2 = diags(np.array([ a2 * np.ones(M-3), di2, a2 * np.ones(M-3)]), [-1, 0, 1], shape=(M-2, M-2)).toarray()

        B1 = np.zeros([M-2, M-2])
        B2 = np.zeros([M-2, M-2])

        D0 = diags(np.array([ np.ones(M-3), -2 * np.ones(M-2), np.ones(M-3)]), [-1, 0, 1], shape=(M-2, M-2)).toarray()
        Dinv = np.asarray(np.linalg.inv(D0))

        b1 = np.zeros(M-2)
        b1[-1] = a1 
        b2 = np.zeros(M-2)
        b2[-1] = a2

        bPsi = np.zeros(M-2)
        bPsi[0] = Psi0

        def B(s, Psi, n):
            diag =  (Psi[n, 1:M-1 ] - Psi[n, 0:M-2 ])
            diag2 =  (Psi[n, 1:M-2 ] - Psi[n, 2:M-1 ])
            PsiMatrix = diags(np.array([ diag , diag2 ]), [0, 1], shape=(M-2, M-2)).toarray()
            if s == 1:
                return  -1 * a1 * PsiMatrix
            if s == -1:
                return a2 * PsiMatrix
        # Set up initial conditions for C

        rho1 = np.zeros(M)
        rho2 = np.zeros(M)
        Psi = np.zeros(M)
        E = np.zeros(M-1)

        Psi[-1] = 0
        Psi[0] = Psi0

        
        #Starting iteration
        for n in range(0, N-1):

             # Update border condition
            g1 = 1 / ( 1 + kf * dxi / ( D1 * kappa ) - (Psi[1]- Psi0))
            A1[0,0] = ( 1 - 2 * a1 + g1 * a1 )

            g2 = 1 / ( 1 + (Psi[1] - Psi[0]))
            A2[0,0] = ( 1 - 2 * a2 + g2 * a2 )


            rho1[1:M-1] = np.matmul(A1, rho1[1:M-1])  + b1 + np.matmul(B(1, Psi, n), rho1[1:M-1])
            rho1[0] = g1 * rho1[n+1, 1]
            rho1[-1] = 1

            rho2[1:M-1] = np.matmul(A2, rho2[1:M-1]) + b2 + np.matmul(B(-1, Psi, n), rho2[1:M-1]) 
            rho2[0] = g2 * rho2[n+1, 1]
            rho2[-1] = 1

            Psi[1:M-1] = np.matmul(Dinv, dxi * (rho2[1:M-1] - rho1[1:M-1]) -bPsi )
            Psi[0] = Psi0
            Psi[-1] = 0
        
            E[0:M-1] = - (Psi[1:M] - Psi[:M-1])/dxi
            
        print("Build Complete")
        self.rho1 = rho1
        self.rho2 = rho2
        self.Psi = Psi
        self.E = E
        
    def remove_points(self, A, n):
        #n is the number of steps to skip
        if n >= 4:
            A = np.delete(A, [1, 2, 3])

        for i in range(0,int(len(A)/4)):
            index = i+n
            A = np.delete(A, [index-2, index-1, index])
        return A
    
    #Cm is the imported analytical solution
    def plot(self, t, f, imageName='complete-diffusion-nernst'):
        
        imageName = imageName + str(t)[-3:]
        Cb = self.Cb
        dtau = self.dtau
        C1 = Cb * self.rho1
        C2 = Cb * self.rho2
        if(f == 'E'):
            func = self.R * self.T * self.E / (self.z * self.F)
            ylabel = r'Electric Field (N/C)'
            title = ''#'Electric Field In The Diffusion Problem With Nernst Interaction.'
        elif(f == 'phi'):
            func = self.R * self.T * self.Psi / (self.z * self.F)
            ylabel = r'Electric Potential (V)'
            title = ''#'Electric Potential In The Diffusion Problem With Nernst Interaction.'
             # this is done to avoid cluttering of numeric points over the analytic solution
        else:
            print("Unkown function. Need something real to plot")
            return -1
        kappa = self.kappa
        D1 = self.D1 
        mw = 4
        fs = 26
        skip = 4

        xi2 = self.xi
        xi2 = kappa * xi2 #* nanometerScale #change the scale of the scale to nanometer

        for i in range(len(t)):
            
            plt.figure(1)
            fig, ax1 = plt.subplots(figsize=(20,16))
            

            color = 'tab:red'
            ax1.tick_params(axis='y', labelcolor=color)
            ax2 = ax1.twinx() 
            color = 'tab:blue'

            plt.title(title, fontsize=fs, fontweight='bold')

            n = int(t[i]/dtau)
        
            ax1.plot(xi2, C1[n, :], 'g^', markersize=mw, label=r'$C_+$,  $\tau ='+str(t[i])+'$')
            ax1.plot(xi2, C2[n, :],'r^', markersize=mw, label=r'$C_-$,  $\tau ='+str(t[i])+'$')
            ax1.legend(loc='upper left', fontsize = fs)
            if f == 'E':
                ax2.plot(xi2[0:self.M-1], func[n, :], 'b^', markersize=mw, color='tab:blue', label=r'$\phi$,  $\tau ='+str(t[i])+'$')
            if f == 'phi':
                ax2.plot(xi2, func[n, :], 'b^', markersize=mw, color='tab:blue', label=r'$\phi$,  $\tau ='+str(t[i])+'$')

            ax1.set_xlabel(r'Distance from the interface plate (nm)', fontsize=fs)
            ax1.set_ylabel(r'Molar Concentration', fontsize=fs)
            ax2.set_ylabel(ylabel, fontsize=fs)
            ax2.tick_params(axis='y', labelcolor=color)
            

            ax2.legend(loc=(0.01,  0.85), fontsize = fs)


            ################################## Plot parameters ##################################
            textstr = '\n'.join((
                r'$V_0=%.2f V$' % Decimal(self.V0),
                r'$k_f=%.2f m/s$' % Decimal(self.kf),
                r'$\kappa=%.2f m^{-1}$' % Decimal(self.kappa),
                r'$C_b=%.2f M$' % Decimal(self.Cb),))


            props = dict(boxstyle='round', facecolor='white', alpha=0.5)
            ax1.text(0.02, 0.8, textstr, transform=ax1.transAxes, fontsize=fs, verticalalignment='top', bbox=props)

            if(len(imageName) > 0):
                plt.savefig('../../img/'+ self.imageName + f + str(i) +'.eps', format='eps', dpi=1000, fontsize=16, fontweight='bold')


            fig.tight_layout()  # otherwise the right y-label is slightly clipped

            plt.show()

    